In [ ]:
import numpy as np
import random
import torch
import re
from transformers import T5Tokenizer, T5ForConditionalGeneration

In [ ]:
ans, qc, spc, corsp, wpc, corwp = 0, 0, 0, 0, 0, 0
wp, sp = False, False

In [ ]:
sp_data = np.load('/content/SP-train.npy', allow_pickle=True)
wp_data = np.load('/content/WP-train.npy', allow_pickle=True)

In [ ]:
print(len(sp_data))
print(len(wp_data))

507
396


In [ ]:
print(sp_data[0])
print(wp_data[0])

{'id': 'SP-0', 'question': 'Mr. and Mrs. Mustard have six daughters and each daughter has one brother. But there are only 9 people in the family, how is that possible?', 'answer': 'Each daughter shares the same brother.', 'distractor1': 'Some daughters get married and have their own family.', 'distractor2': 'Some brothers were not loved by family and moved away.', 'distractor(unsure)': 'None of above.', 'label': 1, 'choice_list': ['Some daughters get married and have their own family.', 'Each daughter shares the same brother.', 'Some brothers were not loved by family and moved away.', 'None of above.'], 'choice_order': [1, 0, 2, 3]}
{'id': 'WP-0', 'question': 'How do you spell COW in thirteen letters?', 'answer': 'SEE O DOUBLE YOU.', 'distractor1': 'COWCOWCOWCOWW', 'distractor2': 'SEE OH DEREFORD', 'distractor(unsure)': 'None of above.', 'label': 1, 'choice_list': ['SEE OH DEREFORD', 'SEE O DOUBLE YOU.', 'COWCOWCOWCOWW', 'None of above.'], 'choice_order': [2, 0, 1, 3]}


In [ ]:
np.random.shuffle(sp_data)
np.random.shuffle(wp_data)

train_ratio = 0.85

sp_train_size = int(len(sp_data) * train_ratio)
wp_train_size = int(len(wp_data) * train_ratio)

sp_train = sp_data[:sp_train_size]
sp_test = sp_data[sp_train_size:]

wp_train = wp_data[:wp_train_size]
wp_test = wp_data[wp_train_size:]

In [ ]:
def format(data):
    id = data['id']
    ques = data['question']
    ch_list = data['choice_list']
    ch_order = data['choice_order']
    ch = [f"{chr(97 + i)}. {ch_list[idx]}" for i, idx in enumerate(ch_order)]
    qtn = f"""Question: {ques}
    {ch[0]}
    {ch[1]}
    {ch[2]}
    {ch[3]}"""
    return qtn, id

In [ ]:
tokenizer = T5Tokenizer.from_pretrained('t5-small')
model = T5ForConditionalGeneration.from_pretrained('t5-small')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.train()

T5ForConditionalGeneration(
  (shared): Embedding(32128, 512)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 512)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=512, out_features=512, bias=False)
              (k): Linear(in_features=512, out_features=512, bias=False)
              (v): Linear(in_features=512, out_features=512, bias=False)
              (o): Linear(in_features=512, out_features=512, bias=False)
              (relative_attention_bias): Embedding(32, 8)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=512, out_features=2048, bias=False)
              (wo): Linear(in_features=2048, out_features=512, bias=False)
              (dropout): Drop

In [ ]:
optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)
loss_fn = torch.nn.CrossEntropyLoss()
num_epochs = 200

In [ ]:
for epoch in range(num_epochs):
  for data in np.concatenate((sp_train, wp_train)):
    wp = False
    sp = False
    qtn, id = format(data)
    if '_' in id:
      continue
    if "WP" in id:
      wp = True
    if "SP" in id:
      sp = True
    qc += 1
    if wp:
      wpc += 1
    if sp:
      spc += 1
    inp = tokenizer(qtn, return_tensors="pt", padding=True, truncation=True).to(device)
    if data['answer'] not in data['choice_list']:
      print(f"Skipping question {id} because answer '{data['answer']}' is not in the list of choices.")
      continue
    lbl = data['choice_list'].index(data['answer'])
    op = model(input_ids = inp.input_ids, attention_mask = inp.attention_mask, labels = torch.tensor([lbl]).unsqueeze(0).to(device))
    loss = op.loss
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    plbl = torch.argmax(op.logits, dim=-1)
    if plbl == lbl:
      ans += 1
      if wp:
        corwp += 1
      if sp:
        corsp += 1
    print(f"Epoch {epoch+1}, Question {qc}: Loss: {loss.item()}")

accuracy = ans / qc
print(f"Total Accuracy: {accuracy:.2f}")
print(f"WP Accuracy: {corwp / wpc:.2f}")
print(f"SP Accuracy: {corsp / spc:.2f}")

Streaming output truncated to the last 5000 lines.
Epoch 181, Question 46604: Loss: 8.344646857949556e-07
Epoch 181, Question 46605: Loss: 0.0
Epoch 181, Question 46606: Loss: 2.4437606043647975e-05
Epoch 181, Question 46607: Loss: 2.1815061700181104e-05
Epoch 181, Question 46608: Loss: 0.0018180761253461242
Epoch 181, Question 46609: Loss: 2.9802276912960224e-06
Epoch 181, Question 46610: Loss: 0.0
Epoch 181, Question 46611: Loss: 0.0
Epoch 181, Question 46612: Loss: 3.576278118089249e-07
Epoch 181, Question 46613: Loss: 2.0861407392658293e-05
Epoch 181, Question 46614: Loss: 2.3841855067985307e-07
Epoch 181, Question 46615: Loss: 5.245071224635467e-05
Epoch 181, Question 46616: Loss: 2.145764938177308e-06
Epoch 181, Question 46617: Loss: 0.00041416651220060885
Epoch 181, Question 46618: Loss: 7.152555099310121e-07
Epoch 181, Question 46619: Loss: 1.6689286894688848e-06
Epoch 181, Question 46620: Loss: 1.5735502529423684e-05
Epoch 181, Question 46621: Loss: 1.5497195136049413e-06
Epoc

In [ ]:
# Train Prediction
print('Train Example:')
print(qtn)
print('Answer:', data['answer'])
print('Predicted:', chr(97 + plbl.item()))

Train Example:
Question: What do you call dissatisfied cranberries?
    a. Blueberries.
    b. Stawberries.
    c. Blackberries
    d. None of above.
Answer: Blueberries.
Predicted: a


In [ ]:
model.eval()

test_ans, test_qc, test_wpc, test_spc, test_corwp, test_corsp = 0, 0, 0, 0, 0, 0

start_token_id = tokenizer.pad_token_id

with torch.no_grad():
    for data in np.concatenate((sp_test, wp_test)):
        wp, sp = False, False
        qtn, id = format(data)
        if '_' in id:
            continue
        if "WP" in id:
            wp = True
        if "SP" in id:
            sp = True

        test_qc += 1
        if wp:
            test_wpc += 1
        if sp:
            test_spc += 1

        inp = tokenizer(qtn, return_tensors="pt", padding=True, truncation=True).to(device)
        if data['answer'] not in data['choice_list']:
            print(f"Skipping question {id} because answer '{data['answer']}' is not in the list of choices.")
            continue

        lbl = data['choice_list'].index(data['answer'])
        decoder_input_ids = torch.full((1, 1), start_token_id, device=inp.input_ids.device)
        op = model(input_ids=inp.input_ids, attention_mask=inp.attention_mask, decoder_input_ids=decoder_input_ids)
        predicted_label = torch.argmax(op.logits, dim=-1).squeeze()

        if predicted_label.item() == lbl:
            test_ans += 1
            if wp:
                test_corwp += 1
            if sp:
                test_corsp += 1

total_accuracy = test_ans / test_qc if test_qc > 0 else 0
wp_accuracy = test_corwp / test_wpc if test_wpc > 0 else 0
sp_accuracy = test_corsp / test_spc if test_spc > 0 else 0

print("Evaluation Results:")
print(f"Total Test Accuracy: {total_accuracy:.2f}")
print(f"WP Test Accuracy: {wp_accuracy:.2f}")
print(f"SP Test Accuracy: {sp_accuracy:.2f}")

Evaluation Results:
Total Test Accuracy: 0.44
WP Test Accuracy: 0.35
SP Test Accuracy: 0.52
